# Import modules

In [2]:
import os
import json
import pprint
import pandas as pd
import numpy as np
from collections import OrderedDict

import mysql.connector
import requests
import json
import time
import argparse
from tqdm import tqdm

# from quickumls import QuickUMLS
from quickumls import get_quickumls_client


# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Set up display area to show dataframe in jupyter qtconsole
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

In [4]:
data = pd.read_csv("./output/subtitles/BERN_annotated_subtitles.csv", sep=";", index_col=["sent_id"])
data['MESHtoCUIs'] = data.MESHtoCUIs.apply(lambda x: eval(x))
print("Number of annotated entities: ", len(data))
data.head()

Number of annotated entities:  25585


,sentence,entity_count,BERN_extracted_ngram,BERN_entity_type,msh_id,MESHtoCUIs
sent_id,,,,,,
25,Gonna give you epinephrine.,1,epinephrine,drug,D004837,"[C0014563, C0521932, C0001636, C0700536, C0699336, C0025133, C0014564, C0205755]"
29,I'll put you on some steroids instead,1,steroids,drug,D013256,"[C0038317, C0007401]"
31,It's called adrenaline.,1,adrenaline,drug,Unknown,[]
41,She went into respiratory distress.,1,respiratory distress,disease,D012127,[C0035220]
42,And you injected her with epinephrine.,1,epinephrine,drug,D004837,"[C0014563, C0521932, C0001636, C0700536, C0699336, C0025133, C0014564, C0205755]"


In [5]:
single_entity_sentences = data.loc[data.index.value_counts() < 2]
print(len(single_entity_sentences))
single_entity_sentences

17948


,sentence,entity_count,BERN_extracted_ngram,BERN_entity_type,msh_id,MESHtoCUIs
sent_id,,,,,,
25,Gonna give you epinephrine.,1,epinephrine,drug,D004837,"[C0014563, C0521932, C0001636, C0700536, C0699336, C0025133, C0014564, C0205755]"
29,I'll put you on some steroids instead,1,steroids,drug,D013256,"[C0038317, C0007401]"
31,It's called adrenaline.,1,adrenaline,drug,Unknown,[]
41,She went into respiratory distress.,1,respiratory distress,disease,D012127,[C0035220]
42,And you injected her with epinephrine.,1,epinephrine,drug,D004837,"[C0014563, C0521932, C0001636, C0700536, C0699336, C0025133, C0014564, C0205755]"
...,...,...,...,...,...,...
204755,"No time, his O2 sats are falling.",1,O2,drug,Unknown,[]
204756,"We have to suck it out here, bedside embolectomy.",1,embolectomy,disease,Unknown,[]
204775,I had a ski injury and painkillers weren't enough and a friend of mine gave me some heroin.,1,heroin,drug,D003932,"[C0011892, C0282128, C0591360, C0591798]"


In [6]:
print("Number of sentences with annotated entities: ", len(data.index.unique()))
print("Number of sentences with one entity: ", len(single_entity_sentences))
print("Number of disease entities: ", len(data.loc[data.BERN_entity_type == 'disease']))
print("Number of drug entities: ", len(data.loc[data.BERN_entity_type == 'drug']))
print("Number of gene entities: ", len(data.loc[data.BERN_entity_type == 'gene']))
print("Number of species entities: ", len(data.loc[data.BERN_entity_type == 'species']))
print("Number of sentences with one disease entity: ", len(single_entity_sentences.loc[single_entity_sentences.BERN_entity_type == 'disease']))
print("Number of sentences with one drug entity: ", len(single_entity_sentences.loc[single_entity_sentences.BERN_entity_type == 'drug']))
print("Number of sentences with one gene entity: ", len(single_entity_sentences.loc[single_entity_sentences.BERN_entity_type == 'gene']))
print("Number of sentences with one species entity: ", len(single_entity_sentences.loc[single_entity_sentences.BERN_entity_type == 'species']))


Number of sentences with annotated entities:  21283
Number of sentences with one entity:  17948
Number of disease entities:  17142
Number of drug entities:  5255
Number of gene entities:  1788
Number of species entities:  1396
Number of sentences with one disease entity:  11496
Number of sentences with one drug entity:  4000
Number of sentences with one gene entity:  1318
Number of sentences with one species entity:  1132


In [7]:
single_disease_sentences = single_entity_sentences.loc[single_entity_sentences.BERN_entity_type == 'disease']
print("number of sentences with one disease entity: ", len(single_disease_sentences))
single_disease_sentences

number of sentences with one disease entity:  11496


,sentence,entity_count,BERN_extracted_ngram,BERN_entity_type,msh_id,MESHtoCUIs
sent_id,,,,,,
41,She went into respiratory distress.,1,respiratory distress,disease,D012127,[C0035220]
54,"If you haven't found an underlying cause for the cardiac arrest by then, I will have to notify our attorneys.",1,cardiac arrest,disease,D006323,"[C0600228, C0018790]"
56,That could manifest with hypo-cardio.,1,hypo-cardio,disease,Unknown,[]
60,"It's no allergic, and allergies don't cause cardiac arrest like this.",1,cardiac arrest,disease,D006323,"[C0600228, C0018790]"
61,Vasculitis?,1,Vasculitis,disease,D014657,[C0042384]
...,...,...,...,...,...,...
204713,I ran out of pain medication.,1,pain,disease,D010146,"[C0030193, C0234238, C0234230, C0458259, C0751407, C0234254, C0458257, C0751408]"
204714,Orbital fracture means your face went into the windshield.,1,Orbital fracture,disease,D009917,"[C0029184, C0162437]"
204721,Doesn't explain the pneumoperitoneum.,1,pneumoperitoneum,disease,D011027,[C0032320]


# CHV entity linking

In [85]:
class Database:
    def __init__(self, DB_HOST, DB_SCHEMA, DB_USER, DB_PASSWORD, DB_PORT):
        try:
            self.connection = mysql.connector.connect(host=DB_HOST,
                                                      database=DB_SCHEMA,
                                                      user=DB_USER,
                                                      password=DB_PASSWORD,
                                                      port=DB_PORT,
                                                      use_pure=True)
            self.cursor = self.connection.cursor()
            logging.info('Successfully connected to MySQL')
        except mysql.connector.Error as e:
            logging.exception("Error while connecting to MySQL", e)

    def __enter__(self):
        return self

    @property
    def get_connection(self):
        return self.connection

    @property
    def get_cursor(self):
        return self._cursor

    def fetch_query(self, query, params=None):
        try:
            self.cursor.execute(query, params, multi=False)
            result = self.cursor.fetchall()
        except Exception as e:
            logging.exception('Query to DB failed: ', e)
        return result

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.connection.is_connected():
            self.connection.close()
            logging.info("MySQL connection is closed.")


def get_semtype(CUIs):
    CUI_semtypes = []
    
    for CUI in CUIs:
        if not CUI:
            continue

        # Retrieve TUI for BERN CUIs
        query_CUI_to_semtype = "SELECT TUI, STY FROM umls.MRSTY WHERE CUI=%(CUI)s;"
        try:
            result = db.fetch_query(query=query_CUI_to_semtype,
                                    params={'CUI': CUI})
            CUI_semtype = {}
            for tuple in result:
                CUI_semtype[tuple[0]] = tuple[1]
            # print('Semantic type: ', CUI_semtype)
        except:
            print('Unable to retrieve semantic type for CUI: ' + CUI)
        CUI_semtypes.append(CUI_semtype)

    return CUI_semtypes


def get_CHV_term(CUIs, semtypes, accepted_semtypes):
    CHV_IDs = []
    CHV_terms = []
    CHV_freq_scores = []
    for i, CUI in enumerate(CUIs):
        if not CUI:
            continue
        
        # print("list(semtypes[i])[0]: ", list(semtypes[i])[0])
        # Query CHV term only if the semantic type of the annotated CUI is one of the UMLS disorder types
        if CUI and (list(semtypes[i])[0] in accepted_semtypes):
            """
            Query CHV terms and their frequency scores via the CHV CODE resulted from the CUI.
            Subsequently, if there are more than one preferred CHV term,
            we could take the SUI of the preferred terms and look up in the MRSAT table
            which one of them has the higher frequency score.
            """
            query_CUI_to_CHV_CODE = """
                                    SELECT DISTINCT umls.MRCONSO.CODE
                                    FROM umls.MRCONSO
                                    WHERE umls.MRCONSO.CUI=%(CUI)s and umls.MRCONSO.SAB='CHV';
                                    """
            try:
                result = db.fetch_query(query_CUI_to_CHV_CODE,
                                        params={'CUI': CUI})
                #print('Result CHV CODE: ', result)
                
                CHV_CODE = []
                for tuple in result:
                    CHV_CODE.append(tuple[0])
            except:
                print('Unable to retrieve CHV CODE for CUI: ' + CUI)
            
            CHV_preferred_term = []
            CHV_freq_score = []
            for n, code in enumerate(CHV_CODE):
                query_CHV_CODE_to_CHV_preferred = """
                                SELECT DISTINCT umls.MRCONSO.STR,umls.MRSAT.ATV
                                FROM umls.MRCONSO
                                INNER JOIN umls.MRSAT ON umls.MRCONSO.SUI=umls.MRSAT.SUI
                                WHERE umls.MRCONSO.CODE=%(CHV_CODE)s and umls.MRCONSO.SAB='CHV' and umls.MRCONSO.TTY='PT' and umls.MRSAT.ATN='FREQUENCY';
                                """
                try:
                    result = db.fetch_query(query_CHV_CODE_to_CHV_preferred,
                                            params={'CHV_CODE': CHV_CODE[n]})
                    #print('CHV term & freq. score: ', result)
                    
                    for tuple in result:
                        CHV_preferred_term.append(tuple[0])
                        CHV_freq_score.append(float(tuple[1]))
                except:
                    print('Unable to retrieve CHV preferred term for CUI: ' + CUI)
            
            CHV_IDs.append(CHV_CODE)
            CHV_terms.append(CHV_preferred_term)
            CHV_freq_scores.append(CHV_freq_score)

    return CHV_IDs, CHV_terms, CHV_freq_scores


def get_exact_match_CUI(BERN_extracted_entity):
    # Retrieve CHV terms and frequency scores through exact string matching with UMLS DB
    query_exact_match_BERN_entity = "SELECT DISTINCT CUI FROM umls.MRCONSO where str=%(BERN_extracted_entity)s;"
    try:
        result = db.fetch_query(
            query_exact_match_BERN_entity,
            params={'BERN_extracted_entity': BERN_extracted_entity})
        exact_match_CUI = []
        for tuple in result:
            exact_match_CUI.append(tuple[0])
        # print('CUI resulted from exact match: ', exact_match_CUI)
    except:
        print('Unable to retrieve CUI for BERN extracted entity: ' +
              BERN_extracted_entity)
    return exact_match_CUI


def qumls_match_entities(accepted_semtypes, BERN_extracted_entity):
    qumls_CUIs = []
    qumls_extracted_ngrams = []
    qumls_matched_terms = []
    qumls_CUI_semtypes = []

    quickumls_results = call_quickumls(accepted_semtypes,
                                       BERN_extracted_entity)
    # print('QuickUMLS results: ', quickumls_results)
    
    if quickumls_results:
        for i in range(len(quickumls_results)):
            for item in quickumls_results[i]:
                quickumls_cui = item['cui']
                # print('QuickUMLS CUI: ', quickumls_cui)
                qumls_extracted_ngram = item['ngram']
                qumls_matched_term = item['term']

                # Retrieve semantic type for QuickUMLS CUIs
                TUI = item['semtypes']
                sem_type = {}
                for tui in TUI:
                    query_semtype = "SELECT SemType FROM umls.MT_SemGroups where TUI=%(tui)s;"
                    try:
                        result = db.fetch_query(query_semtype,
                                                params={'tui': tui})
                        for tuple in result:
                            sem_type[tui] = tuple[0]
                        # print('QUMLS Semantic type: ', sem_type)
                    except:
                        print(
                            'Unable to retrieve semantic type for QUMLS TUI: ' + tui)
                
                qumls_CUIs.append(quickumls_cui)
                qumls_extracted_ngrams.append(qumls_extracted_ngram)
                qumls_matched_terms.append(qumls_matched_term)
                qumls_CUI_semtypes.append(sem_type)
    
    # Retrieve CHV tems and frquency scores for QuickUMLS CUIs
    qumls_CUI_to_CHV_ID, qumls_CHV_terms, qumls_CHV_freq_scores = get_CHV_term(qumls_CUIs, qumls_CUI_semtypes, accepted_semtypes)            

    return qumls_CUIs, qumls_extracted_ngrams, qumls_matched_terms, qumls_CUI_semtypes, qumls_CUI_to_CHV_ID, qumls_CHV_terms, qumls_CHV_freq_scores


def extend_annotated_df(annotated_df, accepted_semtypes):
    extended_df = []
#     for index, row in tqdm(annotated_df.iterrows(), total=annotated_df.shape[0]):
    for index, row in annotated_df.iterrows():
        BERN_CUI_semtypes = get_semtype(row["MESHtoCUIs"])
        BERN_CUI_to_CHV_ID, BERN_CHV_terms, BERN_CHV_freq_scores = get_CHV_term(row["MESHtoCUIs"], BERN_CUI_semtypes, accepted_semtypes)
        
        exact_match_CUIs = get_exact_match_CUI(row["BERN_extracted_ngram"])
        exact_match_CUI_semtypes = get_semtype(exact_match_CUIs)
        exact_match_CUI_to_CHV_ID, exact_match_CHV_terms, exact_match_CHV_freq_scores = get_CHV_term(exact_match_CUIs, exact_match_CUI_semtypes, accepted_semtypes)
        
        qumls_CUIs, qumls_extracted_ngrams, qumls_matched_terms, \
        qumls_CUI_semtypes, qumls_CUI_to_CHV_ID, qumls_CHV_terms, \
        qumls_CHV_freq_scores = qumls_match_entities(accepted_semtypes, row["BERN_extracted_ngram"])
        
        extended_df.append(OrderedDict({"sent_id": index,
                                        "BERN_CUI_semtypes": BERN_CUI_semtypes,
                                        "BERN_CUI_to_CHV_ID": BERN_CUI_to_CHV_ID,
                                        "BERN_CHV_terms": BERN_CHV_terms,
                                        "BERN_CHV_freq_scores": BERN_CHV_freq_scores,
                                        "exact_match_CUIs": exact_match_CUIs,
                                        "exact_match_CUI_semtypes": exact_match_CUI_semtypes,
                                        "exact_match_CUI_to_CHV_ID": exact_match_CUI_to_CHV_ID,
                                        "exact_match_CHV_terms": exact_match_CHV_terms,
                                        "exact_match_CHV_freq_scores": exact_match_CHV_freq_scores,
                                        "qumls_CUIs": qumls_CUIs,
                                        "qumls_extracted_ngrams": qumls_extracted_ngrams,
                                        "qumls_matched_terms": qumls_matched_terms,
                                        "qumls_CUI_semtypes": qumls_CUI_semtypes,
                                        "qumls_CUI_to_CHV_ID": qumls_CUI_to_CHV_ID,
                                        "qumls_CHV_terms": qumls_CHV_terms,
                                        "qumls_CHV_freq_scores": qumls_CHV_freq_scores
                                        }))
        
    extended_df = pd.DataFrame(extended_df).set_index('sent_id')
    extended_df = pd.concat([annotated_df, extended_df], axis=1)
    
    return extended_df

def main():
    accepted_semtypes = {
        # Disorders
        'T020',  # Acquired Abnormality
        'T190',  # Anatomical Abnormality
        'T049',  # Cell or Molecular Dysfunction
        'T019',  # Congenital Abnormality
        'T047',  # Disease or Syndrome
        'T050',  # Experimental Model of Disease
        'T033',  # Finding
        'T037',  # Injury or Poisoning
        'T048',  # Mental or Behavioral Dysfunction
        'T191',  # Neoplastic Process
        'T046',  # Pathologic Function
        'T184',  # Sign or Symptom
    }
        
    df_split = np.array_split(single_disease_sentences, len(single_disease_sentences))
    
    for df in tqdm(df_split):
        extended_df = extend_annotated_df(df, accepted_semtypes)
        # If csv doesn't exist yet, create one, and append to it in the next iterations
        with open('./output/subtitles/subtitles_bern_exact_qumls_oc(length)_th08_jaccard.csv', 'a') as f:
            extended_df.to_csv(f, header=f.tell()==0, sep=';')


Paste this in the terminal to run de QuickUMLS server:

python -m quickumls.server /path/to/umls/eng -t 0.8 -o length -s jaccard -v

In [86]:
if __name__ == "__main__":
    
    # Instantiate QuickUMLS
    quickumls_fp = './umls/eng/'
    # Default values: overlapping_criteria="score", threshold=0.7 , similarity_name="jaccard", window=5
    def call_quickumls(accepted_semtypes, text):
#         matcher = QuickUMLS(quickumls_fp,
#                             overlapping_criteria="length",
#                             threshold=0.8,
#                             similarity_name="jaccard",
#                             accepted_semtypes=accepted_semtypes)
#         quickumls_results = matcher.match(text,
#                                           best_match=True,
#                                           ignore_syntax=False)
        
        matcher = get_quickumls_client()
        quickumls_results = matcher.match(text, best_match=True, ignore_syntax=False)
        
        return quickumls_results
    
    with Database(DB_HOST="myt-research-cluster.cluster-cqjr4dunkiqp.eu-west-1.rds.amazonaws.com",
                  DB_SCHEMA="umls",
                  DB_USER="abc",
                  DB_PASSWORD="123",
                  DB_PORT=3301) as db:
        main()
    
    

INFO:root:Successfully connected to MySQL
100%|██████████| 667/667 [50:27<00:00,  4.54s/it]  
INFO:root:MySQL connection is closed.
